## Analysis of coordinates crossmatch sources compared with Zebrun##

In [1]:
import os
import ipywidgets as widgets
from sqlalchemy import *
from IPython.display import display

Firstly, let's select all the records in our **Be star candidates** table defined by *Sabogal et. al (2005)*

In [2]:
engine = create_engine('sqlite:///BeStarsCatalog.db')
metadata = MetaData()
metadata.reflect(bind=engine)

conn = engine.connect()
sabogal = metadata.tables['Sabogal']
zebrun = metadata.tables['Zebrun']

selection = select([sabogal])
result = conn.execute(selection).fetchall()


Result of the database query for *Sabogal* sources is [**Result Proxy**](http://docs.sqlalchemy.org/en/latest/core/connections.html?highlight=proxy#sqlalchemy.engine.ResultProxy). The next step is to transform result to the *Astropy Table* type

In [3]:
from astropy.io import ascii
from astropy.table import Table

table_data=[]
for row in result:
    table_data.append(dict(row))

sabogal_table = Table(rows=table_data,dtype=('f4','S','S','S','S','f4','f4','f4'))
sabogal_table

B-V,OGLE,Per,Per2,Type,V-I,Vmag,rms
float32,bytes16,bytes6,bytes18,bytes3,float32,float32,float32
0.007,04595836-6925494,None,None,1,0.185,15.949,0.068
-0.028,05013017-6908270,None,None,1,0.126,16.058,0.08
-0.043,05064631-6843094,None,None,1,-0.04,17.666,0.026
-0.117,05064687-6842250,None,None,1,-0.114,16.337,0.028
-0.015,05064836-6837230,None,None,1,0.145,15.573,0.052
-0.085,05065094-7000527,None,None,1,-0.067,16.11,0.013
-0.016,05065274-6834374,None,None,1,0.158,15.698,0.081
-0.094,05070209-6829173,None,None,1,-0.078,16.488,0.026
-0.037,05070440-6842256,None,None,1,0.113,16.465,0.094


Now we have all Be stars candidates with OGLE id analysed by *Sabogal et al. (2005)* in our database. We can also qeury  *Zebrun* catalog database, extract extra informations about stars and their light  curves and store result again to the different *Table* **Zebrun**

In [4]:
selection = select([zebrun])
result = conn.execute(selection).fetchall()

table_data=[]
for row in result:
    table_data.append(dict(row))

zebrun_table = Table(rows=table_data,dtype=('f4','I','I','S','f4','S','I','f4'))

zebrun_table

DEJ2000,Field,Idat,MC,Mag,OGLE,OGLEII,RAJ2000
float32,uint32,uint32,bytes1,float32,bytes16,uint32,float32
-69.43039,15,136,L,15.767,04595836-6925494,5876,74.99316
-69.14083,15,256,L,15.945,05013017-6908270,133526,75.3757
-68.71928,13,244,L,17.598,05064631-6843094,173999,76.69296
-68.70695,13,244,L,16.344,05064687-6842250,173858,76.69528
-68.62305,13,223,L,15.309,05064836-6837230,178889,76.70149
-70.01464,12,301,L,16.19,05065094-7000527,111511,76.71225
-68.57706,13,237,L,15.435,05065274-6834374,183876,76.71975
-68.48814,13,250,L,16.467,05070209-6829173,261089,76.7587
-68.70711,13,252,L,16.245,05070440-6842256,242802,76.76833


Optional step is to convert table to *LaTeX* format, decrease/increase precision of the real numbers and change order of the columns itself.

We can also compare crossmatched sources between OGLE II database and coordinates of Be sources from *Sabogal et al. (2005)*  stored in database file *classification.db* in database table *BeSummary* with original *Zebrun* catalogue.

In [50]:
engine = create_engine('sqlite:///classification.db')
metadata = MetaData()
metadata.reflect(bind=engine)

conn_classification = engine.connect()

classification_result = metadata.tables['BeSummary']
selection_classification = select([classification_result])
result = conn_classification.execute(selection_classification).fetchall()

In [51]:
table_data=[]

for row in result:
    table_data.append(dict(row))

summary_table = Table(rows=table_data,dtype=('S','S','S','S','f4','f4','S','f4','f4'))
summary_table

Per_Det,VCheck,name,oglename,per_1,per_2,type_mennickent,v_i,vmag
str4,str4,str17,str20,float32,float32,str3,float32,float32
None,None,lmc_sc15_i_5876,OGLE04595836-6925494,nan,nan,1,0.185,15.949
None,None,lmc_sc15_i_133526,OGLE05013017-6908270,nan,nan,1,0.126,16.058
None,None,lmc_sc13_i_173999,OGLE05064631-6843094,nan,nan,1,-0.04,17.666
None,None,lmc_sc13_i_173858,OGLE05064687-6842250,nan,nan,1,-0.114,16.337
None,None,lmc_sc13_i_178889,OGLE05064836-6837230,nan,nan,1,0.145,15.573
None,None,lmc_sc12_i_111511,OGLE05065094-7000527,nan,nan,1,-0.067,16.11
None,None,lmc_sc13_i_183876,OGLE05065274-6834374,nan,nan,1,0.158,15.698
None,None,lmc_sc13_i_261089,OGLE05070209-6829173,nan,nan,1,-0.078,16.488
None,None,lmc_sc13_i_242802,OGLE05070440-6842256,nan,nan,1,0.113,16.465


Porovnej crossmatch v databazi *BeSummary* s oficialními publikovanými daty v databázi *Vizier*. We want to know, which *Sabogal* sources published in **Vizier** was not crossmatched by coordinates in our pipeline

In [52]:
for row in zebrun_table:
        ogle_id = 'OGLE'+str(row['OGLE'])
        prefix = str(row['MC']).lower()+'mc_sc'
        ogleii_id = prefix+str(row['Field'])+'_i_'+str(row['OGLEII'])
        select_st = select([classification_result]).where(
            classification_result.c.oglename == ogle_id)

        result= conn_classification.execute(select_st).fetchall()
        if not result:
            print(' %s missing in Sabogal catalogue with OGLE II id %s' %(ogle_id,ogleii_id))
        elif result[0]['name']==ogleii_id:
            pass
        else:
            print('%s have differet OGLEII id %s compare with Zebrun id %s' % (ogle_id,result[0]['name'],ogleii_id))

 OGLE05144218-6915585 missing in Sabogal catalogue with OGLE II id lmc_sc9_i_335785
 OGLE05365595-6953158 missing in Sabogal catalogue with OGLE II id lmc_sc16_i_195859
OGLE05123350-6849228 have differet OGLEII id lmc_sc10_i_278725 compare with Zebrun id lmc_sc9_i_84478
 OGLE05053187-6825516 missing in Sabogal catalogue with OGLE II id lmc_sc13_i_57192
 OGLE05234024-6936239 missing in Sabogal catalogue with OGLE II id lmc_sc5_i_187350
 OGLE05274224-6922279 missing in Sabogal catalogue with OGLE II id lmc_sc3_i_113787
 OGLE05341459-6940395 missing in Sabogal catalogue with OGLE II id lmc_sc1_i_262410
OGLE05060829-6910053 have differet OGLEII id lmc_sc13_i_69027 compare with Zebrun id lmc_sc12_i_104452
 OGLE05450685-7058509 missing in Sabogal catalogue with OGLE II id lmc_sc20_i_14792
 OGLE05172067-6920088 missing in Sabogal catalogue with OGLE II id lmc_sc8_i_325275
 OGLE05182388-6905058 missing in Sabogal catalogue with OGLE II id lmc_sc7_i_221151
 OGLE05195728-6919108 missing in Sabog

We can ask also in opposite way, namely which *Sabogal* sources in crossmatched catalog are not published in **Vizier** catalog

In [53]:
import re

def extract_info(text):
    m = re.search('lmc_sc(\d+)_i_(\d+).*', text)
    if m:
        field = int(m.group(1))
        id_number = int(m.group(2))
    else:
        field = np.nan
        id_number = np.nan
        
    return field,id_number

In [54]:
engine = create_engine('sqlite:///BeStarsCatalog.db')
metadata = MetaData()
metadata.reflect(bind=engine)
zebrun = metadata.tables['Zebrun']

conn = engine.connect()

id = 0
for row in summary_table:
        id += 1
        ogleii_id = str(row['name'])
        ogle_id =  str(row['oglename']).replace('OGLE','')
        field,id_number = extract_info(ogleii_id)
        
        select_st = select([zebrun]).where(
            (zebrun.c.OGLEII ==id_number) &
            (zebrun.c.Field == field))
        
        result= conn.execute(select_st).fetchall()
        if not result:
            print('Crossmatched source with OGLEII id %s does not exist in Zebrun OGLE%s' %(ogleii_id,ogle_id))           


Crossmatched source with OGLEII id lmc_sc10_i_278725 does not exist in Zebrun OGLE05123350-6849228
Crossmatched source with OGLEII id lmc_sc13_i_69027 does not exist in Zebrun OGLE05060829-6910053


From the result, it is clear, that coordinates crossmatch of Zebrun sources with **OGLE** id *OGLE05123350-6849228, OGLE05060829-6910053* led to the fail **OGLEII** identification of the sources. 

Originally, light curves of **Be star candidates** sumarized by *Sabogal et. al (2005)* came from *OGLE* photometry archive. But they are also present in the succesor *OGLE II* archive with more measurements inside. We downloaded all sources from archive using [webscrapOGLE.ipynb](webscrapOGLE.ipynb). Three sources listed below does not contains points (Error: No good DIA photometry for this object)
- lmc_sc20_i_14792.dat 
- lmc_sc8_i_325275.dat 
- lmc_sc7_i_433061.dat